In [2]:
# 0 - kissinfashion
# 1 - instagood
# 2 - beautifuldestinations
# 3 - etdieucrea
# 4 - josecabaco


import numpy as np # linear algebra\n",
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import urllib, cStringIO
import json
import re
import requests
from PIL import Image
%matplotlib inline


email = 'timo261188@gmail.com'
token = '2hTlHfBU6sB6QNWB05cfNS9BigqNjzTY'
live_api_url = 'http://challenges.instagram.unpossib.ly/api/live'
resp = requests.get(live_api_url, auth=(email, token)).json()

with open('dataset.json') as data_file:
    data = json.load(data_file)
    
data_array = []
y = []



In [3]:
# Import all the data from csv's and skip the data processing steps bellow
df = pd.read_csv('dataset_final.csv')
y = pd.read_csv('target.csv')

In [ ]:
for userId in range(0, len(data)):
    print str(userId)
    for postId in range(0, len(data[userId]['posts'])):
        if postId%100 == 0: print str(postId)
        file = cStringIO.StringIO(urllib.urlopen(str(data[userId]['posts'][postId]['instagram']['thumbnail_src'])).read())
        testimg = Image.open(file)
        img = img_to_matrix(testimg)
        data_array.append(flatten_image(img))
        y.append(data[userId]['posts'][postId]['instagram']['likes'])

In [358]:
#Extract average pixel RGB values and their standard deviations for each image

import cv2

image_features = []
y = []

for userId in range(0, len(data)):
    
    for postId in range(0, len(data[userId]['posts'])):
        
        if postId%100 == 0: print str(postId)
            
        req = urllib.urlopen(str(data[userId]['posts'][postId]['instagram']['thumbnail_src']))
        arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
        img = cv2.imdecode(arr,-1) # 'load it as it is'
        (means, stds) = cv2.meanStdDev(img)
        stats = np.concatenate([means, stds]).flatten()
        if len(stats) < 6:
            image_features.append([127, 127, 127, 40, 40, 40]) 
            y.append(data[userId]['posts'][postId]['instagram']['likes']['count'])
        else:
            image_features.append(stats)
            y.append(data[userId]['posts'][postId]['instagram']['likes']['count'])


In [354]:
# Create features linking posts to the accounts

df = pd.DataFame(image_features)
df.columns = [['R','G','B','stdR','stdG','stdB']]

set_arrays = []
for i in range(0, 5): set_arrays.append(len(data[i]['posts']))

accounts = pd.DataFrame()
for i in range(0, 5):
    if i == 0:
        
        x = np.empty(set_arrays[0])
        y = np.empty(sum(set_arrays[1:]))
        x.fill(1)
        y.fill(0)
        z = np.concatenate((x, y))
        accounts['arr'+str(i+1)] = z[:]
        
    if i == 1 or 2 or 3:
        x = np.empty(sum(set_arrays[0:i]))
        y = np.empty(set_arrays[i])
        k = np.empty(sum(set_arrays[i+1:]))
        x.fill(0)
        y.fill(1)
        k.fill(0)
        z = np.concatenate((x, y))
        w = np.concatenate((z, k))
        accounts['arr'+str(i+1)] = w[:]
        
    if i == 4:
        x = np.empty(sum(set_arrays[0:4]))
        y = np.empty(set_arrays[4])
        x.fill(0)
        y.fill(1)
        z = np.concatenate((x, y))  
        accounts['arr'+str(i+1)] = z[:]

df = pd.concat([df, accounts], axis = 1)

In [359]:
# Find most often used words in the captions

words = []

for userId in range(0, len(data)):
    
    for postId in range(0, len(data[userId]['posts'])):
        
        if postId%100 == 0: print str(postId)
        
        try:
            words.append(data[userId]['posts'][postId]['instagram']['caption'].encode('utf-8'))
        except:
            words.append('empty')
            
import collections, re

bagsofwords = [ collections.Counter(re.findall(r'\w+', txt))
            for txt in words]

sumbags = sum(bagsofwords, collections.Counter())

import operator
sorted_bags = sorted(sumbags.items(), key=operator.itemgetter(1))
sorted_bags[-100:] #Look at 100 most frequent

In [ ]:
# Encode popular caption words
word_array = ['credit','watchthisinstagood','instagood','2instagood','instagood10k','yay','nay','suggested','like','photos','love','empty','soir','bdteam']

word_features = []
for row in words:
    feature_array = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    row = row.lower().split()
    for word in row:
        if word in word_array:
            feature_array[word_array.index(word)] = 1
    word_features.append(feature_array)

array = pd.DataFrame(word_features)
array.columns = [['credit','watchthisinstagood','instagood','2instagood','instagood10k','yay','nay','suggested','like','photos','love','empty','soir','bdteam']]
new_df = pd.concat([df, array], axis = 1)

In [360]:
new_df.head() # dataset will look like this

,R,G,B,stdR,stdG,stdB,acc1,acc2,acc3,acc4,...,instagood10k,yay,nay,suggested,like,photos,love,empty,soir,bdteam
0,204.080117,199.022463,209.127378,63.706204,64.031788,53.264128,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,199.460098,200.949404,179.987881,66.830746,62.663154,67.848041,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,153.531646,156.303948,165.212656,77.806725,76.344574,71.112208,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,152.083032,152.576143,159.240066,66.962074,71.132470,73.454708,1.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
4,184.691270,176.546968,172.698362,71.428380,60.898164,59.666383,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Apply cross validation
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Ridge
kf = KFold(n_splits=5, shuffle=True)

clf = Ridge(alpha = 0.1)
scores = cross_val_score(clf, df, y, scoring='r2', cv=kf)

print("R2 = %0.6f" % scores.mean())

R2 = 0.935135


In [240]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = train_test_split(df2[df2.columns[0:-1]], df2[df2.columns[-1]], test_size=0.33, random_state=42)
clf = LinearRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print r2_score(y_test, y_pred)

0.936240286227


In [5]:
#Train the model
clf = Ridge(alpha = 0.1)
clf.fit(df, y)

Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [12]:
#Make submissions
import cv2
import json
import re
import requests
import itertools
from requests.auth import HTTPBasicAuth
import time

post_id = []

while True:
    email = 'timo261188@gmail.com'
    token = '2hTlHfBU6sB6QNWB05cfNS9BigqNjzTY'
    live_api_url = 'http://challenges.instagram.unpossib.ly/api/live'
    resp = requests.get(live_api_url, auth=(email, token)).json()
    
    for i in range(0, 5):
        if len(resp['accounts'][i]['posts']) > 0:
            post_id = resp['accounts'][i]['posts'][0]['instagram']['id'] #save post id

            #process image
            req = urllib.urlopen(str(resp['accounts'][i]['posts'][0]['instagram']['thumbnail_src']))
            arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
            img = cv2.imdecode(arr,-1) # 'load it as it is'
            (means, stds) = cv2.meanStdDev(img)
            stats = np.concatenate([means, stds]).flatten()

            #process account number
            # 0 - kissinfashion
            # 1 - instagood
            # 2 - beautifuldestinations
            # 3 - etdieucrea
            # 4 - josecabaco

            account_array = [0, 0 ,0 ,0 ,0]

            if i == 0:
                account_array[2] = 1
            elif i == 1:
                account_array[3] = 1
            elif i == 2:
                account_array[1] = 1
            elif i == 3:
                account_array[4] = 1
            elif i == 4:
                account_array[0] = 1

            # Encode popular caption words
            caption = resp['accounts'][i]['posts'][0]['instagram']['caption'].encode('utf-8') #get caption
            word_array = ['credit','watchthisinstagood','instagood','2instagood','instagood10k','yay','nay','suggested','like','photos','love','empty','soir','bdteam']

            feature_array = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
            caption = caption.lower().split()
            for word in caption:
                if word in word_array:
                    feature_array[word_array.index(word)] = 1

            #merge features
            merge_features = itertools.chain(stats, account_array, feature_array)
            merge_features = list(merge_features)
            predicted = int(clf.predict(merge_features)[0][0])

            #POST
            r = requests.post("http://challenges.instagram.unpossib.ly/api/submissions", data=json.dumps({"post": str(post_id), "likes": predicted}), auth=HTTPBasicAuth(email, token))
            print r.text
        
    time.sleep(60)
    print "No posts yet..."
    continue


No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...


C:\Anaconda2\envs\gl-env\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


{"success":true,"message":"Submission saved"}

No posts yet...


C:\Anaconda2\envs\gl-env\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


{"success":false,"error":"noSubmission","message":"Nothing was saved. Maybe you've already submitted for this post"}
No posts yet...


...

C:\Anaconda2\envs\gl-env\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


{"success":false,"error":"noSubmission","message":"Nothing was saved. Maybe you've already submitted for this post"}
No posts yet...


C:\Anaconda2\envs\gl-env\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


{"success":false,"error":"noSubmission","message":"Nothing was saved. Maybe you've already submitted for this post"}
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...
No posts yet...


KeyError: 'caption'